# Training and Deploying the Fraud Detection Model

In this notebook, we will take the outputs from the Processing Job in the previous step and use it and train and deploy an XGBoost model. Our historic transaction dataset is initially comprised of data like timestamp, card number, and transaction amount and we enriched each transaction with features about that card number's recent history, including:

- `num_trans_last_10m`
- `num_trans_last_1w`
- `avg_amt_last_10m`
- `avg_amt_last_1w`

Individual card numbers may have radically different spending patterns, so we will want to use normalized ratio features to train our XGBoost model to detect fraud. 

### Imports 

In [1]:
from sklearn.model_selection import train_test_split
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker import image_uris
import pandas as pd
import numpy as np
import sagemaker
import boto3
import io

### Essentials 

In [2]:
LOCAL_DIR = './data'
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'training'

# sagemaker_role = sagemaker.get_execution_role()
sagemaker_role = 'arn:aws:iam::499469212086:role/sagemaker_creditcard'
s3_client = boto3.Session().client('s3')

First, let's load the results of the SageMaker Processing Job ran in the previous step into a Pandas dataframe. 

In [3]:
df = pd.read_csv(f'{LOCAL_DIR}/aggregated/processing_output.csv')
#df.dropna(inplace=True)
df['cc_num'] = df['cc_num'].astype(np.int64)
df['fraud_label'] = df['fraud_label'].astype(np.int64)
df.head()
len(df)

54000

### Split DataFrame into Train & Test Sets

The artifically generated dataset contains transactions from `2020-01-01` to `2020-06-01`. We will create a training and validation set out of transactions from `2020-01-15` and `2020-05-15`, discarding the first two weeks in order for our aggregated features to have built up sufficient history for cards and leaving the last two weeks as a holdout test set. 

In [4]:
training_start = '2020-01-15'
training_end = '2020-05-15'

training_df = df[(df.datetime > training_start) & (df.datetime < training_end)]
test_df = df[df.datetime >= training_end]

test_df.to_csv(f'{LOCAL_DIR}/test.csv', index=False)

Although we now have lots of information about each transaction in our training dataset, we don't want to pass everything as features to the XGBoost algorithm for training because some elements are not useful for detecting fraud or creating a performant model:
- A transaction ID and timestamp is unique to the transaction and never seen again. 
- A card number, if included in the feature set at all, should be a categorical variable. But we don't want our model to learn that specific card numbers are associated with fraud as this might lead to our system blocking genuine behaviour. Instead we should only have the model learn to detect shifting patterns in a card's spending history. 
- Individual card numbers may have radically different spending patterns, so we will want to use normalized ratio features to train our XGBoost model to detect fraud. 

Given all of the above, we drop all columns except for the normalised ratio features and transaction amount from our training dataset.

In [5]:
training_df.drop(['tid','datetime','cc_num','num_trans_last_10m', 'avg_amt_last_10m',
       'num_trans_last_1w', 'avg_amt_last_1w'], axis=1, inplace=True)

/tmp/ipykernel_202185/3217280059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.drop(['tid','datetime','cc_num','num_trans_last_10m', 'avg_amt_last_10m',


The [built-in XGBoost algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) requires the label to be the first column in the training data:

In [6]:
training_df = training_df[['fraud_label', 'amount', 'amt_ratio1','amt_ratio2','count_ratio']]
training_df.head()

,fraud_label,amount,amt_ratio1,amt_ratio2,count_ratio
46,0,362.22,0.535936,0.535936,0.052632
47,0,0.93,0.001401,0.001401,0.052632
48,0,356.44,0.522487,0.522487,0.052632
49,0,30.04,0.039566,0.039566,0.058824
50,0,11.80,0.016441,0.016441,0.055556


In [7]:
train, val = train_test_split(training_df, test_size=0.3)
train.to_csv(f'{LOCAL_DIR}/train.csv', header=False, index=False)
val.to_csv(f'{LOCAL_DIR}/val.csv', header=False, index=False)

In [8]:
!aws s3 cp {LOCAL_DIR}/train.csv s3://{BUCKET}/{PREFIX}/
!aws s3 cp {LOCAL_DIR}/val.csv s3://{BUCKET}/{PREFIX}/

upload: data/train.csv to s3://sagemaker-ap-south-1-499469212086/training/train.csv
upload: data/val.csv to s3://sagemaker-ap-south-1-499469212086/training/val.csv


In [9]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":"100"}

output_path = 's3://{}/{}/output'.format(BUCKET, PREFIX)

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", sagemaker.Session().boto_region_name, "1.2-1")

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role='arn:aws:iam::499469212086:role/sagemaker_creditcard',
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

# define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}".format(BUCKET, PREFIX, 'train.csv'), content_type=content_type)
validation_input = TrainingInput("s3://{}/{}/{}".format(BUCKET, PREFIX, 'val.csv'), content_type=content_type)

# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-05-29-11-18-26-387


2023-05-29 11:18:27 Starting - Starting the training job...
2023-05-29 11:18:41 Starting - Preparing the instances for training...
2023-05-29 11:19:26 Downloading - Downloading input data...
2023-05-29 11:19:41 Training - Downloading the training image...
2023-05-29 11:20:17 Training - Training image download completed. Training in progress...[2023-05-29 11:20:39.080 ip-10-0-64-173.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:De

Ideally we would perform hyperparameter tuning before deployment, but for the purposes of this example will deploy the model that resulted from the Training Job directly to a SageMaker hosted endpoint.

In [10]:
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium',
    serializer=sagemaker.serializers.CSVSerializer(), wait=True)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-05-29-11-21-13-337
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-05-29-11-21-13-337
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-05-29-11-21-13-337


-------!

In [11]:
endpoint_name=predictor.endpoint_name
#Store the endpoint name for later cleanup 
%store endpoint_name
endpoint_name

Stored 'endpoint_name' (str)


'sagemaker-xgboost-2023-05-29-11-21-13-337'

Now to check that our endpoint is working, let's call it directly with a record from our test hold-out set. 

In [12]:
payload_df = test_df.drop(['tid','datetime','cc_num','fraud_label','num_trans_last_10m', 'avg_amt_last_10m',
       'num_trans_last_1w', 'avg_amt_last_1w'], axis=1)
payload = payload_df.head(1).to_csv(index=False, header=False).strip()
payload

'64.96,0.1462232574526806,0.1462232574526806,0.0434782608695652'

In [13]:
float(predictor.predict(payload).decode('utf-8'))

0.00040693936171010137

## Show that the model predicts FRAUD / NOT FRAUD

In [14]:
count_ratio = 0.30
payload = f'1.00,1.0,1.0,{count_ratio:.2f}'
is_fraud = float(predictor.predict(payload).decode('utf-8'))
print(f'With transaction count ratio of: {count_ratio:.2f}, fraud score: {is_fraud:.3f}')

With transaction count ratio of: 0.30, fraud score: 0.797


In [15]:
count_ratio = 0.06
payload = f'1.00,1.0,1.0,{count_ratio:.2f}'
is_fraud = float(predictor.predict(payload).decode('utf-8'))
print(f'With transaction count ratio of: {count_ratio:.2f}, fraud score: {is_fraud:.3f}')

With transaction count ratio of: 0.06, fraud score: 0.002
